# Calcul efficace des coefficients binomiaux

On va commencer par faire un calcul naïf de $\binom{n}{k}$ en utilisant le triangle de Pascal :

* $\binom{n}{k} = 0$ si $k \not\in [|0;n|]$
* $\binom{n}{0} = \binom{n}{n} = 1$
* Si $n > 0, k > 0$ alors $\binom{n}{k} = \binom{n-1}{k-1}+\binom{n-1}{k}$

Écrire une fonction récursive `comb` telle que `comb n k` renvoie $\binom{n}{k}$.

In [6]:
let rec comb n k =
    if n = 0 then 1 else
    if k = 0 || (n=k) then 1 else 
    (comb (n-1) (k-1)) +(comb (n-1) k)

val comb : int -> int -> int = <fun>

In [7]:
assert(List.init 3 (comb 2) = [1;2;1]);
assert(List.init 4 (comb 3) = [1;3;3;1]);
"OK"

- : string = "OK"

On va compter ici le nombre d'appels récursifs. Pour cela, on utilise une variable globale `count` qu'on passera à 0 avant de lancer un calcul et qui sera incrémentée à chaque appel.

Pour pouvoir modifier `count` dans une fonction,il faut la déclarer comme `global` comme on peut le voir dans l'exemple suiant :

In [9]:
let count = ref 0

val count : int ref = {contents = 0}

Modifiez `comb` pour incrémenter la variable `count` à chaque appel :

In [10]:
let rec comb n k =
    incr count;
    if n = 0 then 1 else
    if k = 0 || (n=k) then 1 else 
    (comb (n-1) (k-1)) +(comb (n-1) k)

val comb : int -> int -> int = <fun>

Écrire une fonction `compte` telle que `compte n` calcule $\binom{2n}{n}$ avec la fonction précédente et renvoie la valeur de `count`.

**Attention** il faudra commencer par mettre `count` à 0 avant de lancer le calcul.

In [11]:
let compte n =
    count := 0;
    let _  = comb (2*n) n in !count

val compte : int -> int = <fun>

In [12]:
assert(List.init 10 compte = [1; 3; 11; 39; 139; 503; 1847; 6863; 25739; 97239]);
"OK"

- : string = "OK"

Si on effectue un graphe on peut se rendre compte que le nombre d'appel est exponentiel en $n$.

En traçant `compte n` en fonction de $\binom{2n}{n}$, on peut voir que le nombre d'appel est de l'ordre du coefficient calculé lui-même. Ce qui est assez proche de ce qu'on avait vu pour Fibonacci.

On va maintenant utiliser un dictionnaire comme cache comme dans le cours. Reprendre la fonction précédente pour utiliser le dictionnaire `cache` défini en dehors de la fonction.

Le cache sera réalise par une table de hachage. Si vous avez besoin de vous rafraichir la mémoire : https://v2.ocaml.org/api/Hashtbl.html

La stratégie est la suivante pour calculer `comb n k` :

* est-ce que `(n,k)` a une valeur associée dans le cache ? Si oui, on la renvoie.
* Sinon, on calcule `v = comb n k` avec la stratégie précédente, puis on insère l'association `(n,k) -> v`.

In [18]:
let cache = Hashtbl.create 42

let rec comb n k =
    if Hashtbl.mem cache (n,k) then Hashtbl.find cache (n,k)
    else let v =
        (if n = 0 then 1 else
    if k = 0 || (n=k) then 1 else 
    (comb (n-1) (k-1)) +(comb (n-1) k))
    in Hashtbl.add cache (n,k) v;
    v
        

val cache : (int * int, int) Hashtbl.t = <abstr>
val comb : int -> int -> int = <fun>

In [19]:
(* ATTENTION sauvegardez avant de lancer ce test, car si vous n'avez pas bien fait la fonction
 le navigateur va surement crasher *)
assert(List.init 7 (fun i -> comb (2*(10+i)) (10+i)) = 
    [184756; 705432; 2704156; 10400600; 40116600; 155117520; 601080390]);
"OK"

- : string = "OK"

Ici, utiliser un cache semble être inutile dans la mesure où les lignes du triangle de Pascal vont tout être calculées jusqu'à celle qui nous interesse. Une autre approche consiste à partir de la première ligne du tableau, celle qui contient juste un 1, et à en déduire les autres lignes de proche en proche.
Pour cela, on va définir la fonction `ligne_suivante` qui prend en entrée une ligne du triangle de Pascal comme `[1;2;1]` et renvoie la suivante, ici `[1;3;3;1]`.

In [26]:
let ligne_suivante l =
    let rec aux v l acc =
        match l with
        | [] -> 1::acc
        | t::q -> aux t q ((v+t)::acc)
        in (aux 0 l [])
        

val ligne_suivante : int list -> int list = <fun>

In [27]:
ligne_suivante [1;3;3;1]

- : int list = [1; 4; 6; 4; 1]

In [28]:
assert(ligne_suivante [1;3;3;1] = [1;4;6;4;1]);
"OK"

- : string = "OK"

On peut alors en déduire une nouvelle version non récursive de `comb` qui va appeler `ligne_suivante` dans une 
boucle pour obtenir la ligne contenant le coefficient voulu.

In [31]:
let comb n k =
    let rec aux i l =
        if i = n then l 
        else aux (i+1) (ligne_suivante l)
    in List.nth (aux 0 [1]) k

val comb : int -> int -> int = <fun>

In [32]:
assert(List.init 7 (fun i -> comb (2*(10+i)) (10+i)) = 
    [184756; 705432; 2704156; 10400600; 40116600; 155117520; 601080390]);
"OK"

- : string = "OK"

L'utilisation de plusieurs listes semble peu optimisée de part des copies de listes et de l'usage final de `List.nth`. De plus, on calcule une ligne entière sans en avoir besoin vu que les valeurs aux indices `> k` sont inutiles.

À la place, on va utiliser un tableau qui contient à chaque étape les valeurs de $\binom{i}{j}$ pour $j \le k$ et avec $i$ allant de 0 à n. On remarque qu'on peut calculer cela **en place** de la droite vers la gauche.

Exemple : si on a `[|1;4;6;4;1;?;?;?|]` (les valeurs `?` seront remplies plus tard), on peut passer progressivement à l'étape suivante ainsi :

```
[|1;4;6;4;1;?;?;?|] ->
[|1;4;6;4;1;1;?;?|] ->
[|1;4;6;4;5;1;?;?|] ->
[|1;4;6;10;5;1;?;?|] ->
[|1;4;10;10;5;1;?;?|] ->
[|1;5;10;10;5;1;?;?|]
```

Reprendre la fonction précédente en utilisant un tel tableau.

In [52]:
let comb n k =
    let l = Array.make n 1 in
    for i = 0 to n-1 do
        for j = i downto 1 do
            l.(j) <- l.(j-1) + l.(j)
        done;
    done;
    l.(k)

val comb : int -> int -> int = <fun>

In [53]:
List.init 7 (fun i -> comb (2*(10+i)) (10+i))

- : int list =
[184756; 705432; 2704156; 10400600; 40116600; 155117520; 601080390]

In [54]:
assert(List.init 7 (fun i -> comb (2*(10+i)) (10+i)) = 
    [184756; 705432; 2704156; 10400600; 40116600; 155117520; 601080390]);
"OK"

- : string = "OK"

On dit que la version de `comb` avec un cache est **mémoïsée** et la seconde version avec un tableau est **tabulée**.
Les deux versions sont souvent équivalentes mais ce qui change principalement est la vision qu'on en a :

* avec un cache on part des valeurs qui nous intéressent et on **descend** jusqu'aux cas de base
* avec la version tabulée, on part des cas de base et on cacule en **remontant** jusqu'aux valeurs qui nous intéressent


# Chemins monotones

Il s'agit de https://projecteuler.net/problem=18 et https://projecteuler.net/problem=67 .

On considère le triangle :

```
   3
  7 4
 2 4 6
8 5 9 3
```

et on se pose la question de la plus grande somme qu'on peut faire en partant du sommet et en descendant, en allant à gauche ou à droite. Ici, c'est $3 + 7 + 4 + 9 = 23$.


Le triangle est donné sous la forme d'un tableau :

In [59]:
let t1 = [| [|3|]; [|7;4|]; [|2;4;6|]; [|8;5;9;3|] |]

val t1 : int array array = [|[|3|]; [|7; 4|]; [|2; 4; 6|]; [|8; 5; 9; 3|]|]

Implémenter une solution récursive naïve :

In [61]:
let rec maxsum1 t i j =
    if i = Array.length t - 1 then t.(i).(j)
    else 
        let d = maxsum1 t (i+1) (j+1) in
        let g = maxsum1 t (i+1) (j) in
        t.(i).(j) + (max g d)

val maxsum1 : int array array -> int -> int -> int = <fun>

In [62]:
assert(maxsum1 t1 0 0 = 23);
"OK"

- : string = "OK"

Implémentez les deux stratégies précédentes : mémoïsation et tabulation, pour résoudre efficacement ce problème. On vous donne ici des **grands** triangles. Surtout le deuxième !

In [66]:
let cache = Hashtbl.create 42

let rec maxsum_memo t i j =
    if Hashtbl.mem cache (i,j) then Hashtbl.find cache (i,j)
    else let v = 
    begin
        if i = Array.length t - 1 then t.(i).(j) else 
        let d = maxsum_memo t (i+1) (j+1) in
        let g = maxsum_memo t (i+1) (j) in
        t.(i).(j) + (max g d)
    end in Hashtbl.add cache (i,j) v;
    v

val cache : (int * int, int) Hashtbl.t = <abstr>
val maxsum_memo : int array array -> int -> int -> int = <fun>

In [102]:
let maxsum_tab t =
    let h = Array.length t - 1 in
    let l = Array.copy t.(h) in
    for i = h-1 downto 0 do
        for j = 0 to i do 
            l.(j) <- t.(i).(j) + (max l.(j+1) l.(j))
        done;
    done;
    l.(0)

val maxsum_tab : int array array -> int = <fun>

In [88]:
let t2 = [|
[|75|]
;[|95;64|]
;[|17;47;82|]
;[|18;35;87;10|]
;[|20;4;82;47;65|]
;[|19;1;23;75;3;34|]
;[|88;2;77;73;7;63;67|]
;[|99;65;4;28;6;16;70;92|]
;[|41;41;26;56;83;40;80;70;33|]
;[|41;48;72;33;47;32;37;16;94;29|]
;[|53;71;44;65;25;43;91;52;97;51;14|]
;[|70;11;33;28;77;73;17;78;39;68;17;57|]
;[|91;71;52;38;17;14;91;43;58;50;27;29;48|]
;[|63;66;4;68;89;53;67;30;73;16;69;87;40;31|]
;[|4;62;98;27;23;9;70;98;73;93;38;53;60;4;23|]
|]

val t2 : int array array =
  [|[|75|]; [|95; 64|]; [|17; 47; 82|]; [|18; 35; 87; 10|];
    [|20; 4; 82; 47; 65|]; [|19; 1; 23; 75; 3; 34|];
    [|88; 2; 77; 73; 7; 63; 67|]; [|99; 65; 4; 28; 6; 16; 70; 92|];
    [|41; 41; 26; 56; 83; 40; 80; 70; 33|];
    [|41; 48; 72; 33; 47; 32; 37; 16; 94; 29|];
    [|53; 71; 44; 65; 25; 43; 91; 52; 97; 51; 14|];
    [|70; 11; 33; 28; 77; 73; 17; 78; 39; 68; 17; 57|];
    [|91; 71; 52; 38; 17; 14; 91; 43; 58; 50; 27; 29; 48|];
    [|63; 66; 4; 68; 89; 53; 67; 30; 73; 16; 69; 87; 40; 31|];
    [|4; 62; 98; 27; 23; 9; 70; 98; 73; 93; 38; 53; 60; 4; 23|]|]

In [103]:
maxsum_tab t2

- : int = 1074

In [104]:
assert (maxsum_memo t2 0 0 = 1074);
assert (maxsum_tab t2 = 1074);
"OK"

- : string = "OK"